In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import lumicks.pylake as lk

%matplotlib inline

# Twistable Worm-Like-Chain Fitting

In this notebook, we analyze force extension data of DNA over its full range of structural transitions under mechanical stress (0 - 60 pN). The twistable worm-like chain model (tWLC) takes twisting deformations on the DNA double helix into account. Here we will use this model to describe the mechanical response of DNA at such high forces.

Download the files with [`download_from_doi()`](https://lumicks-pylake.readthedocs.io/en/stable/examples/twlc_fitting/../../_api/lumicks.pylake.download_from_doi.html#lumicks.pylake.download_from_doi):

In [ ]:
lk.download_from_doi("10.5281/zenodo.12668419", "data")

Let’s load and plot the data first:

In [ ]:
file = lk.File("data/twlc_data.h5")
fd_curve = file.fdcurves["FD_1_control_forw"]
fd_curve.plot_scatter()

## Set up a basic model first

We clearly see that the force starts levelling out at high forces in the data. We’ll need something rather complex in order to capture this behavior. The twistable worm-like chain (tWLC) model can describe the untwisting behavior of DNA that becomes apparent in the 30-60 pN force range. However, the model’s complexity also incurs some challenges.

Parameter estimation typically begins from an initial guess, and if this initial guess is bad, it can get stuck at an estimated set of parameters that are suboptimal, a so-called local optimum. One way to mitigate this, is to start with better initial values.

In this notebook, we fit the region before the force begins levelling out (i.e. 30 pN) first with a regular worm-like chain model and then use those estimates as initial guesses to fit the tWLC model.

Depending on your experiments, small offsets can be present in the data. For instance, the bead diameter may vary slightly from experiment to experiment, or the force may have experienced some drift. We incorporate an offset in both distance and force to compensate for small offsets that may exist in the data. Let’s set up the Odijk worm-like chain model using the function [`ewlc_odijk_force()`](https://lumicks-pylake.readthedocs.io/en/stable/examples/twlc_fitting/../../_api/lumicks.pylake.ewlc_odijk_force.html#lumicks.pylake.ewlc_odijk_force) and create the fit:

In [ ]:
m_odijk = lk.ewlc_odijk_force("DNA").subtract_independent_offset() + lk.force_offset("DNA")
fit_odijk = lk.FdFit(m_odijk)

Considering that this model only describes the force-extension behaviour at low forces (0.1 - 30 pN), we have to extract the data that is relevant to us. We can obtain this data from the force-distance curve as follows:

In [ ]:
force = fd_curve.f.data
distance = fd_curve.d.data

We only wish to use the forces below 30, so we filter the data according to this requirement:

In [ ]:
mask = force < 30
distance = distance[mask]
force = force[mask]

Now we are ready to add this data to the fit using [`add_data()`](https://lumicks-pylake.readthedocs.io/en/stable/examples/twlc_fitting/../../_api/lumicks.pylake.FdFit.html#lumicks.pylake.FdFit.add_data), but first, we must constrain the distance offset to help the fitting, as this provides a lot of additional freedom in the model:

In [ ]:
fit_odijk.add_data("Inverted Odijk", force, distance)
fit_odijk["DNA/d_offset"].upper_bound = 0.01
fit_odijk["DNA/d_offset"].lower_bound = -0.01

And [`fit()`](https://lumicks-pylake.readthedocs.io/en/stable/examples/twlc_fitting/../../_api/lumicks.pylake.FdFit.html#lumicks.pylake.FdFit.fit) the model:

In [ ]:
fit_odijk.fit()

## Set up the twistable worm like chain model

By default, the [`twlc_distance()`](https://lumicks-pylake.readthedocs.io/en/stable/examples/twlc_fitting/../../_api/lumicks.pylake.twlc_distance.html#lumicks.pylake.twlc_distance) model provided with pylake outputs the distance as a function of force. However, we typically want to fit force as a function of distance. To achieve this, we can invert the model using its [`invert()`](https://lumicks-pylake.readthedocs.io/en/stable/examples/twlc_fitting/../../_api/lumicks.pylake.fitting.model.Model.html#lumicks.pylake.fitting.model.Model.invert) function at the cost of slowing down the fit. Alternatively, we have a faster way of achieving this in pylake, by using the dedicated [`twlc_force()`](https://lumicks-pylake.readthedocs.io/en/stable/examples/twlc_fitting/../../_api/lumicks.pylake.twlc_force.html#lumicks.pylake.twlc_force) model:

In [ ]:
m_dna = lk.twlc_force("DNA").subtract_independent_offset() + lk.force_offset("DNA")
fit_twlc = lk.FdFit(m_dna)

## Load the full data into the model

In the plot showing the data, we could see that there is a small transition event at the end of the Fd curve. The model will not be able to capture this behaviour and therefore it is best to remove this section prior to fitting:

In [ ]:
force = fd_curve.f.data
distance = fd_curve.d.data
mask = distance < 2.88
distance = distance[mask]
force = force[mask]

Now we can load the data into the model:

In [ ]:
fit_twlc.add_data("Twistable WLC", force, distance)

We could add more datasets in a similar manner, but in this example, we only fit a single model. Let’s load the parameters from our previous fit to use them as initial guesses for this one. We also fix the twist rigidity and critical force to values from literature (analogous to Broekmans et al. “DNA twist stability changes with magnesium (2+) concentration.” Physical Review Letters 116, 258102 (2016)):

In [ ]:
fit_twlc.update_params(fit_odijk)

# Fix twist rigidity and critical force to literature values.
fit_twlc["DNA/C"].value = 440
fit_twlc["DNA/C"].fixed = True
fit_twlc["DNA/Fc"].value = 30.6
fit_twlc["DNA/Fc"].fixed = True

## Fit the model

Considering that the tWLC model is more difficult to evaluate, this may take a while. This is also why we choose to enable verbose output:

In [ ]:
fit_twlc.fit(verbose=2)
plt.show()

## Plotting the results

After fitting we can plot our results and print our parameters by invoking [`plot()`](https://lumicks-pylake.readthedocs.io/en/stable/examples/twlc_fitting/../../_api/lumicks.pylake.FdFit.html#lumicks.pylake.FdFit.plot) and [`params`](https://lumicks-pylake.readthedocs.io/en/stable/examples/twlc_fitting/../../_api/lumicks.pylake.FdFit.html#lumicks.pylake.FdFit.params) respectively:

In [ ]:
fit_twlc.plot()
plt.xlabel("Distance [$\\mu$m]")
plt.ylabel("Force [pN]");

We can also show the parameters:

In [ ]:
fit_twlc.params

These seem to agree well with what’s typically found for dsDNA.